In [26]:
%load_ext autoreload
%autoreload 2
from utils import*
from preprocessing import preprocessing
from feature_extraction import extract_features
from training import train_svm
from performance import performance_analysis

Preprocessing

In [ ]:
for i in range(0, 6):
    print(i)
    path_of_the_directory='../data/men/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/men/'+str(i)+'/'
    ind=0
    for filename in os.listdir(path_of_the_directory):
        if(ind>20):
            break
        ind=ind+1
        img = cv2.imread(path_of_the_directory+str(filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preprocessing(img, gamma=True, close=True)
        cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

    path_of_the_directory='../data/women/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/women/'+str(i)+'/'
    ind=0
    for filename in os.listdir(path_of_the_directory):
        if(ind>20):
            break
        ind=ind+1
        img = cv2.imread(path_of_the_directory+str( filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preprocessing(img, gamma=True, close=True)
        cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

Feature Extraction

In [25]:
number_of_clusters = 8
feature_vector_men, expected_class_men = extract_features('../outputs/preprocessing_output/', num_of_clusters=number_of_clusters, debug=False)

c:\Users\mohab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [31]:
# creating df object with columns specified   
csv = pd.DataFrame(feature_vector_men, columns =list(range(number_of_clusters))) 

#Adding Lables to the Data Frame

csv.insert(0, "class", expected_class_men)
print(csv)


csv.to_csv (r'../outputs/features.csv', index = None)

     class    0    1    2    3    4    5    6    7
0        0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0
1        0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2        0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3        0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4        0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
..     ...  ...  ...  ...  ...  ...  ...  ...  ...
309      5  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
310      5  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
311      5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
312      5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
313      5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[314 rows x 9 columns]


Training

In [33]:
# Read from CVS File
df = pd.read_csv('../outputs/features.csv')
# print(df)


classes=df.loc[:,"class"]
classes = classes.to_numpy()
# print(classes)
# print(np.shape(classes))


features=df.loc[:, df.columns != 'class']
features = features.to_numpy()
# print(features)
# print(np.shape(features))

Model Training

In [35]:
prediction, actual = train_svm(features, classes)
# print(prediction)
# print(actual)

Accuracy

In [36]:
accuracy = performance_analysis(prediction,actual)
print(accuracy)

0.6666666666666666


[[1. 2. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
